In [1]:
import sys, os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from tactic_gen.train_codellama import get_trainer
from tactic_gen.train_codellama import get_tokenizer
from transformers import CodeLlamaTokenizer

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conf = {
    "data_path": "/home/ubuntu/coq-modeling/data/data-basic",
    "model_name": "codellama/CodeLlama-7b-hf",
    "output_dir": "/home/ubuntu/coq-modeling/models/test",

    # Training Args
    "max_seq_len": 512,
    "max_input_len": 448,
    "per_device_train_batch_size": 4,
    "learning_rate": 1.41e-5,
    "num_train_epochs": 1,
    "peft_lora_r": 64,
    "peft_lora_alpha": 16,

    # Evaluation Args
    "eval_steps": 500,
    "eval_accumulation_steps": 1,
    "per_device_eval_batch_size": 2,
    "num_eval_examples": 2000, # Evaluation would take ~2 hours each time w/o limiting this

    # Logging Args
    "logging_steps": 10,
    "save_steps": 100,
    "save_total_limit": 5,
}

In [3]:
tokenizer = get_tokenizer(conf)

In [4]:
input_ids = tokenizer("hi there bobbie", return_tensors="pt")["input_ids"].to("cuda")

In [7]:
tokenizer.decode(input_ids[0])

'<s> hi there bobbie</s>'

In [3]:
trainer = get_trainer(conf)



Building Training Config...
gradient_accumulation_steps not found in configuration. Defaulting to 2
max_steps not found in configuration. Defaulting to -1


Retrieving Model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.88s/it]




Retrieving Tokenizer...


Constructing Dataset...


Building Trainer...


/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [4]:
trainer.train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 164135
})

In [5]:
trainer.tokenizer.decode(trainer.train_dataset[1]["input_ids"])

'<s> Theorem andb_eq_orb : forall (b c : bool), (andb b c = orb b c) -> b = c.\n  intros.\n  destruct b,c.<THM-SEP>H: true && true = true || true\n\ntrue = true<GOAL-SEP>H: true && false = true || false\n\ntrue = false<GOAL-SEP>H: false && true = false || true\n\nfalse = true<GOAL-SEP>H: false && false = false || false\n\nfalse = false\n<TACTIC>\n\n  {</s>'